# QCTRL basics

### Introduction
The most important thing to keep in mind while starting to use QCTRL is that it doesn't allow users to perform calculations on their personal machine. This means that every time we want the result of some operation we need to ask their servers to compute it.

We can tell their servers which calculations to make by creating a "graph". You can think of a graph as a recepie that only QCTRL knows how to read.

The typical qctrl workflow looks like this:
- Create graph
- Assign operations to the graph
- Ask QCTRL to compute the graph
- Extract the results

The results obtained this way are not part of their recepie anymore and can be used as regualr python objects, like `numpy` arrays.

IMPORTANT: QCTRL will only be able to return results that have a `name` field. See the first example for a clarification.

### References
[Documentation](https://docs.q-ctrl.com/boulder-opal/get-started)

[API reference](https://docs.q-ctrl.com/boulder-opal/references/qctrl/Qctrl.html)

## First steps
Before continuing make sure you have installed all the packages

In [ ]:
pip install qctrl

### Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import qctrlvisualizer
from qctrl import Qctrl

# Apply Q-CTRL style to plots created in pyplot.
plt.style.use(qctrlvisualizer.get_qctrl_style())

### Start a session
After importing we need to create a Boulder Opal senssion. Boulder Opal is the name of the product/library we are using. It's a good idea to keep this piece of code in a cell separated from the rest, because, each time it runs, it establishes a connection to QCTRL servers and it may take a few seconds.

The first time you run this cell you'll be asked to log in. Just follow thier instructions.

In [3]:
# Start a Boulder Opal session.
qctrl = Qctrl()

## Basic examples

### Learn about the general workflow with Fock states
In this example we'll guide you through the general workflow and we'll do it by computing a simple Fock state.

Consider a cavity and assume we want to describe it up to the dimension `c_dim = 5`. Our goal is to find the expression of Fock state $|1\rangle$.

In [6]:
# Choose Hilber space simulation size
c_dim = 5

# Create a new graph
graph = qctrl.create_graph()

# Add the fock state to the graph
psi = graph.fock_state(c_dim, 1)

You would expect to be able to use `psi` as a regular variable, but that's not the case. For example `psi` can NOT be printed, because right now we have only create a recepie for the calculation of the Fock state, we don't have the actual result.

In order to find the result we need to give a name to this calculation, which can be done in different ways. Note that all the names withing a graph must be unique.

In [8]:
# Add a name to a pre-existing graph node
psi.name = "psi1"

# Assign the name directly during the definition
psi = graph.fock_state(c_dim, 1, name="psi2")

Now we need to send our recepie (`graph`) to QCTRL servers and wait for them to give us our result. Graphs can have many operations in them and we usually are not interested in retrieving the result for all of them. That's why we need to give names to the operations that we care about: this way we can specifically ask QCTRL to give us only those results. We specify these names in the `output_node_names` parameter.

In [10]:
# Send graph to QCTRL and retrieve the result
result = qctrl.functions.calculate_graph(
    graph=graph,
    output_node_names=["psi1"]
)

  0%|          | 0/100 [00:00<?, ?it/s]

Your task calculate_graph (action_id="1674605") has completed.


Now we can access the result of our computation as a regular `numpy` array.

In [14]:
# Access the result
result.output["psi1"]["value"]

array([0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])

This is what the whole code looks like by putting everything together:

In [16]:
# Choose Hilber space simulation size
c_dim = 5

# Create a new graph
graph = qctrl.create_graph()

# Add the fock state to the graph
psi = graph.fock_state(c_dim, 1, name="psi")

# Send graph to QCTRL and retrieve the result
result = qctrl.functions.calculate_graph(
    graph=graph,
    output_node_names=["psi"]
)

# Access the result
result.output["psi"]["value"]

  0%|          | 0/100 [00:00<?, ?it/s]

Your task calculate_graph (action_id="1674607") has completed.


array([0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])

### Learn about operations between graph nodes
In the example above `psi` is a graph node and we know that before computing the graph its value can't be accessed. Does this mean that we need to create a graph for every step of our calculations? Thankfully no. We can make graph nodes talk between each other as we would for `numpy` arrays, but it's important to keep in mind that they are not.

In the following example we'll create the coherent state with $\alpha = 1$ from the vacuum state $|0\rangle$ by applying the displacement operator $\hat{D}(1)$.

In [18]:
# Choose Hilber space simulation size
c_dim = 5

# Create a new graph
graph = qctrl.create_graph()

# Define the vacuum state
vacuum = graph.fock_state(c_dim, 0)

# Define the displacement operator D(1)
alpha = 1
D1 = graph.displacement_operator(alpha, c_dim)

We have defined 2 graph nodes called `vacuum` and `D1`. Now we want to apply the displacement operator to the vacuum state. We can do it with a simple matrix multiplication.

**IMPORTANT**: when multiplying a matrix and a vector we need to give the vector a temporary extra dimension (`[:,None]`). This means that the result of the operation will also have an extra dimension that we need to remove (`[:,0]`).

In [20]:
# Compute the final state by explicitly calling the graph.matmul function
psi = graph.matmul(D1, vacuum[:,None])

# Alternative way to compute the final state using the @ sintactic sugar
psi = D1 @ vacuum[:,None]

# Remove the extra dimension from the result
psi = psi[:,0]

QCTRL gives us the possibility to short the lengthy `graph.matmul` notation with just `@`. Note the two versions call the same exact function, but in the second case it's hidden for readability purposes.

A third way to do the same is to use the `*` to execute the multiplication. This way there is no need to define a temporary dimension:

In [ ]:
psi = D1 * vacuum

Now we just need to retrieve the results.

In [21]:
# Assign a name to the grah node
psi.name = "psi"

# Send graph to QCTRL and retrieve the result
result = qctrl.functions.calculate_graph(
    graph=graph,
    output_node_names=["psi"]
)

# Access the result
result.output["psi"]["value"]

  0%|          | 0/100 [00:00<?, ?it/s]

Your task calculate_graph (action_id="1674609") has completed.


array([0.60655682+3.87710697e-16j, 0.60628133-1.82145965e-15j,
       0.4303874 -1.40859546e-15j, 0.24104351-8.32667268e-16j,
       0.14552147-4.51028104e-16j])

So by putting everything together we get

In [22]:
# Choose Hilber space simulation size
c_dim = 5

# Create a new graph
graph = qctrl.create_graph()

# Define the vacuum state
vacuum = graph.fock_state(c_dim, 0)

# Define the displacement operator D(1)
alpha = 1
D1 = graph.displacement_operator(alpha, c_dim)

# Compute the final state
psi = D1 @ vacuum[:,None]
psi = psi[:,0]
psi.name = "psi"

# Send graph to QCTRL and retrieve the result
result = qctrl.functions.calculate_graph(
    graph=graph,
    output_node_names=["psi"]
)

# Access the result
result.output["psi"]["value"]

  0%|          | 0/100 [00:00<?, ?it/s]

Your task calculate_graph (action_id="1674610") has completed.


array([0.60655682+3.87710697e-16j, 0.60628133-1.82145965e-15j,
       0.4303874 -1.40859546e-15j, 0.24104351-8.32667268e-16j,
       0.14552147-4.51028104e-16j])

### Relationship between `numpy` and `graph`
It should be noted that `numpy` arrays and matrices can be used in operations with graph nodes, but the result will always be a graph node. This means that the result of the operation will be accessible only when the graph is computed.

In this example we will multiply a the identity matrix $I$ by the vector representing Fock state $|0\rangle$ only using `numpy` and a mix of QCTRL and `numpy`.

In [29]:
# Only numpy
I = np.eye(3)
fock0 = np.array([1,0,0], dtype="complex")
I * fock0

array([[1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j]])

In [28]:
# Mixed numpy - QCTRL

I = np.eye(3)

graph = qctrl.create_graph()
fock0 = graph.fock_state(3,0)

op = I * fock0
op.name = "op"

result = qctrl.functions.calculate_graph(
    graph=graph,
    output_node_names=["op"]
)

result.output["op"]["value"]

  0%|          | 0/100 [00:00<?, ?it/s]

Your task calculate_graph (action_id="1674611") has completed.


array([[1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j]])